## All Techniques Of Hyper Parameter Optimization
    1.GridSearchCV
    2.RandomizedSearchCV
    3.Bayesian Optimization -Automate Hyperparameter Tuning (Hyperopt)
    4.Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)
    5.Optuna- Automate Hyperparameter Tuning
    6.Genetic Algorithms (TPOT Classifier)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('Feature_Selection.csv')
df.head()

,PriorDefault_t,YearsEmployed,CreditScore,Income,Approved
0,1,3.04,6,560.000000,1
1,1,1.50,0,824.000000,1
2,1,3.75,5,3.000000,1
3,1,1.71,0,322.159652,1
4,1,2.50,0,322.159652,1


In [3]:
#### Independent And Dependent features
X=df.drop('Approved',axis=1)
y=df['Approved']

In [4]:
#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

In [5]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [6]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(f'confusion matrix : {confusion_matrix(y_test,prediction)}')
print(f'Accuracy Score : {accuracy_score(y_test,prediction)}')
print(classification_report(y_test,prediction))

confusion matrix : [[60 15]
 [11 52]]
Accuracy Score : 0.8115942028985508
              precision    recall  f1-score   support

           0       0.85      0.80      0.82        75
           1       0.78      0.83      0.80        63

    accuracy                           0.81       138
   macro avg       0.81      0.81      0.81       138
weighted avg       0.81      0.81      0.81       138



# Manual Hyperparameter Tuning

In [7]:
### Manual Hyperparameter Tuning
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[68  7]
 [11 52]]
0.8695652173913043
              precision    recall  f1-score   support

           0       0.86      0.91      0.88        75
           1       0.88      0.83      0.85        63

    accuracy                           0.87       138
   macro avg       0.87      0.87      0.87       138
weighted avg       0.87      0.87      0.87       138



# Randomized Search Cv

In [8]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [9]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   41.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [10]:
rf_randomcv.best_params_

{'n_estimators': 200,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 560,
 'criterion': 'gini'}

In [11]:
rf_randomcv

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [12]:
best_random_grid=rf_randomcv.best_estimator_

In [13]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[64 11]
 [10 53]]
Accuracy Score 0.8478260869565217
Classification report:               precision    recall  f1-score   support

           0       0.86      0.85      0.86        75
           1       0.83      0.84      0.83        63

    accuracy                           0.85       138
   macro avg       0.85      0.85      0.85       138
weighted avg       0.85      0.85      0.85       138



# GridSearch CV

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)

{'criterion': ['gini'], 'max_depth': [560], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [8, 9, 10, 11, 12], 'n_estimators': [0, 100, 200, 300, 400]}


In [15]:
#### Fit the grid_search to the data
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  4.1min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [560],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [8, 9, 10, 11, 12],
                         'n_estimators': [0, 100, 200, 300, 400]},
             verbose=2)

In [16]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=560, max_features='sqrt', min_samples_split=10)

In [17]:
best_grid=grid_search.best_estimator_

In [18]:
best_grid

RandomForestClassifier(max_depth=560, max_features='sqrt', min_samples_split=10)

In [19]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[64 11]
 [10 53]]
Accuracy Score 0.8478260869565217
Classification report:               precision    recall  f1-score   support

           0       0.86      0.85      0.86        75
           1       0.83      0.84      0.83        63

    accuracy                           0.85       138
   macro avg       0.85      0.85      0.85       138
weighted avg       0.85      0.85      0.85       138



### Automated Hyperparameter Tuning
Automated Hyperparameter Tuning can be done by using techniques such as

- Bayesian Optimization
- Gradient Descent
- Evolutionary Algorithms
    
#### Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

- Objective Function = defines the loss function to minimize.
- Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
- Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [20]:
!pip install hyperopt

In [21]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [22]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.choice('max_depth',[10, 120, 230, 340, 450, 560, 670, 780, 890, 1000]),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000])
    }

In [23]:

def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [24]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [16:02<00:00, 12.03s/trial, best loss: -0.8584602784602783]


{'criterion': 0,
 'max_depth': 4,
 'max_features': 1,
 'min_samples_leaf': 0.007105237760531388,
 'min_samples_split': 0.6304517767273082,
 'n_estimators': 6}

In [25]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 200, 1: 400, 2: 600, 3: 800, 4: 1000, 5:1200, 6:1400,7:1600,8:1800,9:2000}



print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

entropy
sqrt
1400


In [26]:
best['min_samples_leaf']

0.007105237760531388

In [27]:

trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[70  5]
 [16 47]]
0.8478260869565217
              precision    recall  f1-score   support

           0       0.81      0.93      0.87        75
           1       0.90      0.75      0.82        63

    accuracy                           0.85       138
   macro avg       0.86      0.84      0.84       138
weighted avg       0.85      0.85      0.85       138



## Genetic Algorithms
Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [28]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [29]:
!pip install tpot

In [30]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.8421003914101344
Generation 2 - Current best internal CV score: 0.8439252089283825
Generation 3 - Current best internal CV score: 0.8439252089283825
Generation 4 - Current best internal CV score: 0.8439252089283825
Generation 5 - Current best internal CV score: 0.8457235798159315
Best pipeline: RandomForestClassifier(RandomForestClassifier(input_matrix, criterion=gini, max_depth=780, max_features=log2, min_samples_leaf=8, min_samples_split=10, n_estimators=1800), criterion=gini, max_depth=230, max_features=auto, min_samples_leaf=8, min_samples_split=10, n_estimators=800)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [31]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

0.8768115942028986


## Optimize hyperparameters of the model using Optuna
The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [32]:
!pip install optuna

In [33]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()



In [34]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2020-08-06 01:25:24,243] Finished trial#0 with value: 0.8239882790845016 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1420, 'max_depth': 97.42866513199431}. Best is trial#0 with value: 0.8239882790845016.
[I 2020-08-06 01:25:25,354] Finished trial#1 with value: 0.6878415300546448 with parameters: {'classifier': 'SVC', 'svc_c': 1518.2931242552634}. Best is trial#0 with value: 0.8239882790845016.
[I 2020-08-06 01:25:29,672] Finished trial#2 with value: 0.8239783796626278 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1720, 'max_depth': 11.06727846545538}. Best is trial#0 with value: 0.8239882790845016.
[I 2020-08-06 01:25:29,848] Finished trial#3 with value: 0.6860200364298725 with parameters: {'classifier': 'SVC', 'svc_c': 643.0507856895659}. Best is trial#0 with value: 0.8239882790845016.
[I 2020-08-06 01:25:32,778] Finished trial#4 with value: 0.8239783796626278 with parameters: {'classifier': 'RandomForest', 'n_estimators': 940, 'max_depth': 77.

[I 2020-08-06 01:33:36,574] Finished trial#72 with value: 0.8239882790845016 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1180, 'max_depth': 27.0965426092884}. Best is trial#12 with value: 0.8276114674902986.
[I 2020-08-06 01:33:41,248] Finished trial#73 with value: 0.823968480240754 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1570, 'max_depth': 10.785179438098279}. Best is trial#12 with value: 0.8276114674902986.
[I 2020-08-06 01:33:45,004] Finished trial#74 with value: 0.8239783796626278 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1420, 'max_depth': 18.921043200581245}. Best is trial#12 with value: 0.8276114674902986.
[I 2020-08-06 01:33:49,959] Finished trial#75 with value: 0.8257998732874 with parameters: {'classifier': 'RandomForest', 'n_estimators': 1680, 'max_depth': 15.62231838337615}. Best is trial#12 with value: 0.8276114674902986.
[I 2020-08-06 01:33:55,004] Finished trial#76 with value: 0.8257998732874 with paramete

Accuracy: 0.8276114674902986
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 1300, 'max_depth': 13.246273436648117}


In [35]:
trial

FrozenTrial(number=12, value=0.8276114674902986, datetime_start=datetime.datetime(2020, 8, 6, 1, 26, 39, 493348), datetime_complete=datetime.datetime(2020, 8, 6, 1, 26, 42, 706757), params={'classifier': 'RandomForest', 'n_estimators': 1300, 'max_depth': 13.246273436648117}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': LogUniformDistribution(high=100, low=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=12, state=TrialState.COMPLETE)

In [36]:
study.best_params

{'classifier': 'RandomForest',
 'n_estimators': 1300,
 'max_depth': 13.246273436648117}

In [37]:
rf=RandomForestClassifier(n_estimators=330,max_depth=30)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=30, n_estimators=330)

In [38]:
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[63 12]
 [16 47]]
0.7971014492753623
              precision    recall  f1-score   support

           0       0.80      0.84      0.82        75
           1       0.80      0.75      0.77        63

    accuracy                           0.80       138
   macro avg       0.80      0.79      0.79       138
weighted avg       0.80      0.80      0.80       138



In [39]:
import pickle

 # save the model to disk
filename = 'finalized_model_RF.sav'
pickle.dump(rf_randomcv, open(filename, 'wb'))